In [12]:
[<Measure>]
type deg

let inline internal toSpecialJulianDayNumber (date: DateTime) =
    date.ToOADate() - 36525.

/// Normalize angle to be between 0 and 360, including 0 but excluding 360.
let inline internal normalize (angle: float<deg>) =
     angle - floor ((float angle) / 360.) * 360.<deg>

let inline internal sinDeg (angle: float<deg>) =
    sin (angle * Math.PI / 180.<deg>)

let inline internal cosDeg (angle: float<deg>) =
    cos (angle * Math.PI / 180.<deg>)
/// Long ascending node.
let N julian = 125.1228<deg> - 0.0529538083<deg> * julian |> normalize
/// Arg. of perigee.
let w julian = 318.0634<deg> + 0.1643573223<deg>  * julian |> normalize
/// Mean anomaly.
let M julian = 115.3654<deg> + 13.0649929509<deg> * julian |> normalize
/// Inclination.
let i = 5.1454<deg>
/// Mean distance.
let a =  60.2666
/// Eccentricity.
let e = 0.054900

/// Iterative computation of eccentric anomaly.
let E0 julian =
    M julian +
    (180.<deg> / Math.PI) * e * sinDeg (M julian) *
    (1. + e * cosDeg (M julian)) |> normalize

let rec ecc last julian =
    let now = last -
                (last - (180.<deg> / Math.PI) * e * sinDeg (last) - M julian) /
                (1. - e * cosDeg (last) )  |> normalize
    printf "%f %f\n" now last
    match abs (now - last) with
    | diff when diff <  0.002<deg> -> now
    | _ -> ecc now julian

let E5 julian = ecc (E0 julian) julian

/// X and Y coordinates
let x julian = a * (cosDeg (E5 julian) - e)
let y julian = a * sqrt (1. - e ** 2.) * sinDeg (E5 julian)

/// Distance in earth radii.
let r julian = sqrt( (x julian) ** 2. + (y julian) ** 2. )
/// True anomaly
let v julian = 180.<deg> / Math.PI * atan2 (y julian) (x julian) |> normalize

/// Ecliptic coordinates.
let xeclip julian = r julian * ( cosDeg (N julian) * cosDeg(v julian + w julian) - sinDeg(N julian) * sinDeg(v julian + w julian) * cosDeg i )
let yeclip julian = r julian * ( sinDeg (N julian) * cosDeg(v julian + w julian) + cosDeg(N julian) * sinDeg(v julian + w julian) * cosDeg i )

/// Ecliptic longitude.
let computeLongitude julian = 180.<deg> / Math.PI * atan2 (yeclip julian) (xeclip julian) |> normalize

let day = toSpecialJulianDayNumber (System.DateTime (1990, 4, 19))

E5 day


262.973461 262.968870
262.973461 262.973461


262.97346092942985